# Tuning Fufi hyperparameters with W&B Sweep 🧹 🐶
The idea is automatically tuning hyperparameters with the sweep by the means of a `grid `search.
In order not to spend an eternity doing this thing, we start with a` random` search, and then put a `grid `search on it.

## W&B Setup
🪄 Install `wandb` library and login

Start by installing the library and logging in to your free account.

In [1]:
!pip install wandb -qU
# Log in to your W&B account
import wandb
wandb.login()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.4 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Installing libraries 📚

In [1]:
!pip install gym==0.25.2
#needed from March
!pip install numpy==1.23.5

In [2]:
!pip install stable_baselines3 -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 31.2 MB/s eta 0:00:00


## Setting things up for the environment 🌍 🪖

In [3]:
#cloning Fufi repo from git
!git clone https://github.com/Gaianeve/gym-TheRealFufi.git
#installing things
!pip install /content/gym-TheRealFufi

Cloning into 'gym-TheRealFufi'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 39 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 15.16 KiB | 7.58 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Processing ./gym-TheRealFufi
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
# Enter the environment directory
%cd /content/gym-TheRealFufi
# Actually importing the library for our environment
import gym_TheRealFufi

/content/gym-TheRealFufi


In [5]:
#get back to content directory so I save everything there
%cd ..
!pwd

/content
/content


## importing libraries and functions 📚


In [7]:
#libraries
import argparse
import random
import time
from distutils.util import strtobool
import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.distributions.categorical import Categorical
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

## Log needed function from files 📡
Loading files directly from git, so I don't have to upload it by hand.

In [8]:
#get files from git
!git clone https://github.com/Gaianeve/The_real_Fufi.git

fatal: destination path 'The_real_Fufi' already exists and is not an empty directory.


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
!pwd
%cd The_real_Fufi/Software
from environment import vectorize_env
from agent_class import Agent
from agent_utils import anneal, collect_data, GAE, PPO_train_agent, evaluate_agent


/content
/content/The_real_Fufi/Software


In [10]:
#back to content directory just to be sure
%cd ..
%cd ..

/content/The_real_Fufi
/content


In [12]:
#import main function from file
!pwd
%cd The_real_Fufi/Software
from sweep_main import train_main, parse_args, get_default_hyperparameters_and_names

#back to content directory
%cd ..
%cd ..

/content
/content/The_real_Fufi/Software


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (sweep_main.py, line 43)

## Define the sweep 📑
Standard default parameters from *stable baseline* work pretty well, so I guess a vanilla grid search would be more than enough.

*   `ent_coef ` ➡ Discrete uniform distribution on integers.
*  ` num_envs`  ➡ list of values to try out
*   `learning_rate` ➡  see ent_coef
*  ` num_steps`  ➡ list of values to try out

In [ ]:
import numpy as np
import random

# Define sweep config
sweep_configuration = {
    "method": "random",
    "name": "sweep_Fufi",
    "metric": {"goal": "maximize", "name": "sum_episodes"},
    "parameters": {
       "num_steps": {"values": [128, 256, 512, 2048, 4096, 8192]},
       "ent_coef": {'distribution': 'uniform', 'min': 0.01,'max': 1},
       "num_envs" : {"values": [2,4,8,16,32]},
       "learning_rate":  {'distribution': 'uniform', 'min': 1.5e-7,'max': 1.5e-4}
    },
}


In [ ]:
#print the result
import pprint
pprint.pprint(sweep_configuration)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'sum_episodes'},
 'name': 'sweep_Fufi',
 'parameters': {'ent_coef': {'distribution': 'uniform', 'max': 1, 'min': 0.01},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.00015,
                                  'min': 1.5e-07},
                'num_envs': {'values': [2, 4, 8, 16, 32]},
                'num_steps': {'values': [128, 256, 512, 2048, 4096, 8192]}}}


## Run main with the sweep 🏃 🧹
The `wandb.sweep` function initializes the sweep using the configuration. The `wandb.agent `function runs the sweep, executing the `sweep_main` function for each set of parameters.

📚 **Handling Parameters in Script**: In `sweep_main`, `wandb.init()` initializes a run. The script updates the args with the parameters from the sweep `(wandb.config)`, which are then passed to the main function.

📚 **Note**: Added `if __name__ == "__main__":` This ensures that main is called only when the script is executed directly, not when imported as a module.

In [ ]:
import wandb
if __name__ == "__main__":
    sweep_id = wandb.sweep(sweep=sweep_configuration, project="Fufi_sweep")  # Set up the sweep

    #get default configuration
    default_config  = get_default_hyperparameters_and_names()

    def sweep_train():
      print(default_config)
      parse_args(default_config)
      train_main(default_config)


/usr/local/lib/python3.10/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)


Create sweep with ID: q7flojfh
Sweep URL: https://wandb.ai/cartpole_maria_gaia/Fufi_sweep/sweeps/q7flojfh


In [ ]:
 wandb.agent(sweep_id, function=sweep_train, count = 50)

wandb: Agent Starting Run: 2dy9nk6e with config:
wandb: 	ent_coef: 0.3096027342320947
wandb: 	learning_rate: 1.6650237413872244e-05
wandb: 	num_envs: 16
wandb: 	num_steps: 256
ERROR:wandb.agents.pyagent:Run 2dy9nk6e errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 307, in _run_job
    self._function()
  File "<ipython-input-11-37bcd94df82f>", line 5, in sweep_train
    print(default_config)
NameError: name 'default_config' is not defined

wandb: ERROR Run 2dy9nk6e errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 307, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-11-37bcd94df82f>", line 5, in sweep_train
wandb: ERROR     print(default_config)
wandb: ERROR NameError: name 'default_config' is not defined
wandb: ERROR 
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: 